In [26]:
%run dataset.ipynb
%run model.ipynb
%run utils.ipynb
%run rouge.ipynb

In [2]:
torch.cuda.set_device(1)

In [5]:
g = NeuralSumToEmbedding(glove_filename='/home/ramkishore.s/meta/glove/glove.6B.200d.txt')
g.vectors = g.vectors.reshape(-1, g.dim)

In [17]:
train = DataHandler()
train.load_dump('../data/processed/cnn/training.pkl')
train.pad(padding_idx=g.padding_idx)
train.make_batches(20)

In [23]:
test = DataHandler()
test.load_dump('../data/processed/cnn/test.pkl')
test.pad(padding_idx=g.padding_idx)
test.make_batches(20)

In [11]:
summary_scores = pickle.load(open('../data/processed/cnn/refresh_scores.pkl', 'rb'))

In [16]:
m = EncoderDecoder(torch.cuda.FloatTensor(g.vectors), word_emb_size=g.dim, 
                       sen_emb_size=350, doc_emb_size=600, sen_len=50, batch_size=20, output_dim=2,
                       reverse=True)
m.cuda()

EncoderDecoder(
  (sentenceEncoder): KimCNN(
    (embeddings): Embedding(400002, 200)
    (convolutions): ModuleList(
      (0): Conv2d(1, 50, kernel_size=(1, 200), stride=(1, 1))
      (1): Conv2d(1, 50, kernel_size=(2, 200), stride=(1, 1))
      (2): Conv2d(1, 50, kernel_size=(3, 200), stride=(1, 1))
      (3): Conv2d(1, 50, kernel_size=(4, 200), stride=(1, 1))
      (4): Conv2d(1, 50, kernel_size=(5, 200), stride=(1, 1))
      (5): Conv2d(1, 50, kernel_size=(6, 200), stride=(1, 1))
      (6): Conv2d(1, 50, kernel_size=(7, 200), stride=(1, 1))
    )
  )
  (documentEncoder): LSTM(350, 600, batch_first=True)
  (documentDecoder): LSTM(350, 600, batch_first=True)
  (classifier): Linear(in_features=600, out_features=2, bias=True)
)

In [24]:
iterations = 10
if not os.path.exists('../temp'):  os.mkdir('../temp')

In [ ]:
for i in range(iterations):
    train_refresh(m, batches=train.batches, lines=train.lines, scores=summary_scores, 
                          iterations=1, max_=15, single_sample=True)
    
    get_summaries(m, test.lines, test.batches, 
                      doc_folder='../data/parsed_data/cnn/test/documents/', 
                      write_folder='../temp/', 
                      output_dim=2, summary_len=3)
    
    scores = computeRouge('../data/parsed_data/cnn/test/summaries/', '../temp/')
    
    torch.save(m.state_dict(), 'R1_f1_' + str(scores[0]) + '.params')